# Classificação de Variedades de Grãos de Trigo (Seeds Dataset)

## Metodologia CRISP-DM

Este projeto segue a metodologia CRISP-DM (Cross-Industry Standard Process for Data Mining) para desenvolver um modelo de aprendizado de máquina capaz de classificar variedades de grãos de trigo com base em suas características físicas, utilizando o **Seeds Dataset** do UCI Machine Learning Repository.

### 1. Entendimento do Negócio

**Problema:** Em cooperativas agrícolas de pequeno porte, a classificação dos grãos é realizada manualmente por especialistas, o que é demorado e sujeito a erros. A automação desse processo pode aumentar a eficiência e a precisão.

**Objetivo:** Desenvolver um modelo de classificação de aprendizado de máquina para distinguir três variedades de grãos de trigo (Kama, Rosa e Canadian) com base em 7 características geométricas.

### 2. Entendimento dos Dados

O conjunto de dados contém 210 amostras de grãos de trigo, 70 de cada variedade. As características (atributos) são:

1.  **Area** (Área)
2.  **Perimeter** (Perímetro)
3.  **Compactness** (Compacidade)
4.  **Length of kernel** (Comprimento do Núcleo)
5.  **Width of kernel** (Largura do Núcleo)
6.  **Asymmetry coefficient** (Coeficiente de Assimetria)
7.  **Length of kernel groove** (Comprimento do Sulco do Núcleo)
8.  **Target** (Variedade do Grão: 1=Kama, 2=Rosa, 3=Canadian)

### 3. Preparação dos Dados (Análise Exploratória e Pré-processamento)

#### 3.1. Carregamento e Inspeção Inicial

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuração para visualização
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

# Carregar o dataset a partir do arquivo TXT baixado
# O arquivo TXT não tem cabeçalho e os dados são separados por espaços.
# As colunas são: Area, Perimeter, Compactness, Length of kernel, Width of kernel, Asymmetry coefficient, Length of kernel groove, Variety (target)
column_names = ['Area', 'Perimeter', 'Compactness', 'Length_of_kernel', 'Width_of_kernel', 'Asymmetry_coefficient', 'Length_of_kernel_groove', 'Variety']
df = pd.read_csv('seeds_dataset.txt', sep='\s+', header=None, names=column_names, skipinitialspace=True)

# Mapear os valores numéricos para os nomes das variedades
variety_map = {1: 'Kama', 2: 'Rosa', 3: 'Canadian'}
df['Variety_Name'] = df['Variety'].map(variety_map)

# Exibir as primeiras linhas
print('Primeiras 5 linhas do Dataset:')
print(df.head())

# Informações gerais sobre o dataset
print('\nInformações do Dataset:')
df.info()

# Verificar a contagem de classes
print('\nContagem de Variedades:')
print(df['Variety_Name'].value_counts())

#### 3.2. Estatísticas Descritivas

In [ ]:
# Calcular estatísticas descritivas
print('Estatísticas Descritivas:')
print(df.drop(columns=['Variety', 'Variety_Name']).describe().T)

#### 3.3. Visualização da Distribuição das Características

In [ ]:
# Histograma para todas as features
df.drop(columns=['Variety', 'Variety_Name']).hist(bins=15, figsize=(15, 10), layout=(3, 3))
plt.suptitle('Distribuição das Características (Histogramas)', y=1.02)
plt.tight_layout()
plt.show()

# Boxplots para todas as features, agrupados por variedade
features = df.drop(columns=['Variety', 'Variety_Name']).columns
plt.figure(figsize=(18, 12))
for i, feature in enumerate(features):
    plt.subplot(3, 3, i + 1)
    sns.boxplot(x='Variety_Name', y=feature, data=df)
    plt.title(f'Boxplot de {feature} por Variedade')
    plt.xlabel('Variedade')
    plt.ylabel(feature)
plt.tight_layout()
plt.show()

#### 3.4. Análise de Correlação e Relações entre Características

In [ ]:
# Matriz de Correlação
correlation_matrix = df.drop(columns=['Variety', 'Variety_Name']).corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Correlação entre as Características')
plt.show()

# Gráficos de Dispersão (Pair Plot) para identificar relações
sns.pairplot(df.drop(columns=['Variety']), hue='Variety_Name', palette='viridis')
plt.suptitle('Gráficos de Dispersão das Características por Variedade', y=1.02)
plt.show()

#### 3.5. Tratamento de Valores Ausentes e Verificação de Tipos de Dados

In [ ]:
# Verificação de valores ausentes
print('Valores Ausentes por Coluna:')
print(df.isnull().sum())

# Verificação de tipos de dados (já feita com df.info(), mas repetindo para clareza)
print('\nTipos de Dados:')
print(df.dtypes)

#### 3.6. Padronização/Normalização das Características

Modelos baseados em distância (como KNN e SVM) são sensíveis à escala das features. A análise exploratória (Estatísticas Descritivas e Boxplots) mostrará que as features têm escalas diferentes. Portanto, a padronização (StandardScaler) ou normalização (MinMaxScaler) será necessária. A padronização é geralmente preferida para algoritmos que assumem distribuição normal, mas o MinMaxScaler pode ser útil para preservar a forma da distribuição. Para este projeto, utilizaremos a **Padronização (StandardScaler)**, que transforma os dados para ter média 0 e desvio padrão 1, o que é um bom ponto de partida para a maioria dos algoritmos de ML.

**Nota:** A padronização será aplicada após a separação dos dados em treino e teste (na próxima fase) para evitar *data leakage*.

### 4. Modelagem (Implementação e Comparação de Algoritmos)

#### 4.1. Separação dos Dados e Padronização

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Separar features (X) e target (y)
X = df.drop(columns=['Variety', 'Variety_Name'])
y = df['Variety']

# Separar em conjuntos de treinamento e teste (70% treino, 30% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Padronizar as features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print('Dados separados e padronizados com sucesso.')
print(f'Shape de X_train_scaled: {X_train_scaled.shape}')
print(f'Shape de X_test_scaled: {X_test_scaled.shape}')

#### 4.2. Implementação e Avaliação dos Modelos Base

Serão implementados e comparados os seguintes algoritmos de classificação:

1.  **K-Nearest Neighbors (KNN)**
2.  **Support Vector Machine (SVM)**
3.  **Random Forest**
4.  **Logistic Regression**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

models = {
    'KNN': KNeighborsClassifier(),
    'SVM': SVC(random_state=42),
    'RandomForest': RandomForestClassifier(random_state=42),
    'LogisticRegression': LogisticRegression(random_state=42, max_iter=200)
}

results = {}

print('Treinando e avaliando modelos base...')

for name, model in models.items():
    # Treinamento
    if name in ['KNN', 'SVM']:
        # Modelos baseados em distância usam dados escalados
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        # Modelos de árvore e regressão logística podem usar dados não escalados, mas usaremos escalados para consistência
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        
    # Avaliação
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    
    results[name] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'Confusion Matrix': confusion_matrix(y_test, y_pred),
        'Classification Report': classification_report(y_test, y_pred, target_names=variety_map.values(), output_dict=True)
    }
    
    print(f'\nModelo {name} treinado e avaliado.')
    print(f'Acurácia: {accuracy:.4f}')
    
# Comparação dos resultados
comparison_df = pd.DataFrame({
    'Model': results.keys(),
    'Accuracy': [res['Accuracy'] for res in results.values()],
    'Precision (Macro)': [res['Precision'] for res in results.values()],
    'Recall (Macro)': [res['Recall'] for res in results.values()],
    'F1-Score (Macro)': [res['F1-Score'] for res in results.values()]
})

print('\nComparação de Desempenho dos Modelos Base:')
print(comparison_df.sort_values(by='F1-Score (Macro)', ascending=False).reset_index(drop=True))

# Visualização das Matrizes de Confusão
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()
for i, (name, res) in enumerate(results.items()):
    sns.heatmap(res['Confusion Matrix'], annot=True, fmt='d', cmap='Blues', cbar=False, ax=axes[i],
                xticklabels=variety_map.values(), yticklabels=variety_map.values())
    axes[i].set_title(f'Matriz de Confusão - {name}')
    axes[i].set_xlabel('Predito')
    axes[i].set_ylabel('Verdadeiro')
plt.tight_layout()
plt.show()

### 5. Avaliação (Otimização dos Modelos)

Com base nos resultados da comparação, o modelo com melhor desempenho será selecionado para otimização de hiperparâmetros usando `GridSearchCV`.

In [ ]:
from sklearn.model_selection import GridSearchCV

# Selecionar o melhor modelo (assumindo que o SVM ou Random Forest será o melhor)
# Vamos otimizar o SVM, que geralmente se beneficia da padronização e é um bom candidato para este dataset.
best_model_name = 'SVM'
best_model = SVC(random_state=42)

# Definir o grid de hiperparâmetros para o SVM
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.01, 0.1, 1],
    'kernel': ['rbf', 'linear']
}

print(f'Iniciando otimização de hiperparâmetros para o modelo {best_model_name}...')

grid_search = GridSearchCV(best_model, param_grid, cv=5, scoring='f1_macro', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

print('Melhores hiperparâmetros encontrados:')
print(grid_search.best_params_)

# Treinar o modelo otimizado
optimized_model = grid_search.best_estimator_
y_pred_optimized = optimized_model.predict(X_test_scaled)

# Avaliar o modelo otimizado
accuracy_opt = accuracy_score(y_test, y_pred_optimized)
f1_opt = f1_score(y_test, y_pred_optimized, average='macro')
conf_matrix_opt = confusion_matrix(y_test, y_pred_optimized)
class_report_opt = classification_report(y_test, y_pred_optimized, target_names=variety_map.values())

print(f'\nDesempenho do Modelo Otimizado ({best_model_name}):')
print(f'Acurácia: {accuracy_opt:.4f}')
print(f'F1-Score (Macro): {f1_opt:.4f}')
print('\nRelatório de Classificação:\n', class_report_opt)

# Visualização da Matriz de Confusão Otimizada
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix_opt, annot=True, fmt='d', cmap='Greens', cbar=False,
            xticklabels=variety_map.values(), yticklabels=variety_map.values())
plt.title(f'Matriz de Confusão - {best_model_name} Otimizado')
plt.xlabel('Predito')
plt.ylabel('Verdadeiro')
plt.show()

### 6. Implantação (Interpretação e Insights)

#### 6.1. Interpretação dos Resultados

A análise final será realizada com base nos resultados obtidos, focando em:

1.  **Comparação Pré e Pós-Otimização:** Houve melhoria significativa no desempenho após a otimização de hiperparâmetros?
2.  **Análise da Matriz de Confusão:** Quais variedades são mais difíceis de classificar (mais erros de confusão)?
3.  **Importância das Features (para Random Forest):** Quais características geométricas são mais relevantes para a classificação?

#### 6.2. Importância das Features (para o Random Forest Base)

In [ ]:
# Re-treinar o Random Forest (modelo base) para extrair a importância das features
rf_model = models['RandomForest']
rf_model.fit(X_train_scaled, y_train)

feature_importances = pd.Series(rf_model.feature_importances_, index=X.columns)
feature_importances = feature_importances.sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=feature_importances.values, y=feature_importances.index, palette='viridis')
plt.title('Importância das Features (Random Forest)')
plt.xlabel('Score de Importância')
plt.ylabel('Feature')
plt.show()

print('Importância das Features:\n', feature_importances)